In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
emotions = load_dataset("GerindT/mini_amazon_sentimental",)
emotions = emotions['test'].train_test_split(test_size=0.2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
emotions = emotions.remove_columns(["title", "content","label","score"])
emotions = emotions.rename_column("sentiment", "label")

emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 320000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 80000
    })
})

In [ ]:
# Map sentiment values to numerical labels
sentiment_mapping = {
    "Perfect": 3,
    "Neutral": 1,
    "Negative": 0,
    "Satisfied": 2
}

# Apply the mapping to the 'label' column
emotions = emotions.map(lambda example: {'label': sentiment_mapping[example['label']]})

Map:   0%|          | 0/320000 [00:00<?, ? examples/s]

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
train_ds = emotions["train"]
train_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 320000
})

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True,
                               batch_size=None)

Map:   0%|          | 0/320000 [00:00<?, ? examples/s]

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
num_labels = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=AutoModelForSequenceClassification.from_pretrained(model_ckpt,
         num_labels = num_labels).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions,
                           references = labels)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(

    output_dir="distilbert-emotion-mini-amazon",
    num_train_epochs = 0.5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = True,
    report_to = "none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = emotions_encoded["train"],
    eval_dataset = emotions_encoded["test"],
    tokenizer = tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.230300,0.218986,0.913800


TrainOutput(global_step=10000, training_loss=0.2907, metrics={'train_runtime': 3520.0868, 'train_samples_per_second': 45.453, 'train_steps_per_second': 2.841, 'total_flos': 8661685648232064.0, 'train_loss': 0.2907, 'epoch': 0.5})

In [ ]:
preds_output = trainer.predict(emotions_encoded["test"])


In [ ]:
preds_output.metrics

{'test_loss': 0.21898557245731354,
 'test_accuracy': 0.9138,
 'test_runtime': 510.9292,
 'test_samples_per_second': 156.577,
 'test_steps_per_second': 9.786}

In [ ]:
trainer.push_to_hub(commit_message="Training completed!")


CommitInfo(commit_url='https://huggingface.co/ncoskun/distilbert-emotion-mini-amazon/commit/4d8252ce009a819684f99b09407f06aae0bd5ffe', commit_message='Training completed!', commit_description='', oid='4d8252ce009a819684f99b09407f06aae0bd5ffe', pr_url=None, pr_revision=None, pr_num=None)